# Notebook 4: Mountain Wave Test Case

This notebook will go through an example of nonhydrostatic flow over a mountain. The setup is described in the paper avaliable at this link: https://doi.org/10.1002/qj.603. 

The new features in this example are related to the introduction of the mountain. This requires that we deform the mesh so that the bottom of the domain follows the specified mountain profile. We then have the issue that when we calculate the initial balanced conditions, we no longer have a flat bottom on which to specify the required bottom boundary condition. We solve this problem by applying the boundary condition at the top and then applying a shift so that the bottom boundary condition is as required. The final new feature is the sponge layer which is required at the top of the domain to damp vertically propagating waves so that they are not reflected back into the domain by the nonphysical impermeable upper boundary.

We begin by importing the required functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, VectorFunctionSpace,
                       PeriodicIntervalMesh, ExtrudedMesh, SpatialCoordinate,
                       exp, pi, cos, Function, conditional, Mesh)

INFO     Running /Users/JS1075/firedrake_dec24/lib/python3.12/site-packages/ipykernel_launcher.py -f /private/var/folders/f0/llvlmlb50qg6mmlxs8m6d6d00000gp/T/tmp53ehmk08.json --HistoryManager.hist_file=:memory:


Define the time-step size and the duration of the simulation. 

In [2]:
dt = 5.0
tmax = 9000.

Create a periodic interval mesh in the horizontal and extrude this in the vertical dimension to create a two dimensional mesh with vertically aligned columns. The extrusion is necessary because we create the compatible finite element function spaces by taking tensor products of the horizontal and vertical function spaces.

In [3]:
nlayers = 70    # number of horizontal layers
ncolumns = 180  # number of columns
domain_width = 144000.
base_mesh = PeriodicIntervalMesh(ncolumns, domain_width)

domain_height = 35000.
ext_mesh = ExtrudedMesh(base_mesh, layers=nlayers,
                        layer_height=domain_height/nlayers)

Now we transform the mesh to follow the defined terrain. First we set the parameters describing the mountain width, height and position. Then we create an expression `xexpr` for the coordinates of the deformed mesh. The first component of the coordinates is unchanged but the second is replaced by a new coordinate that transitions smoothly from following the mountain profile at the bottom of the domain to flat layers at a height specified by the `zh` parameter. We then create a new vector function space, `Vc`, that has quadratic basis functions (required to represent the deformed mesh coordinates). We interpolate our `xexpr` to a new `Function` defined on `Vc` and then create a new mesh with these coordinates which we can use to set up our domain.

In [4]:
a = 1000.    # half-width of mountain
hm = 1.      # height of mountain
xc = domain_width/2.   # x position of mountain centre
xz = SpatialCoordinate(ext_mesh)  # get coordinates of undeformed mesh
zs = hm*a**2/((xz[0]-xc)**2 + a**2)  # this defines the mountain as a function of x

zh = 5000.   # height at which mesh layers become flat again
xexpr = as_vector([xz[0], conditional(xz[1] < zh, xz[1] + cos(0.5*pi*xz[1]/zh)**6*zs, xz[1])])

# create new mesh using deformed coordinates
Vc = VectorFunctionSpace(ext_mesh, "DG", 2)
new_coords = Function(Vc).interpolate(xexpr)
mesh = Mesh(new_coords)
mesh._base_mesh = base_mesh  # Force new mesh to inherit original base mesh
domain = Domain(mesh, dt, family="CG", degree=1)

Now we set up the physical parameters, taking the defaults described [here](https://www.firedrakeproject.org/gusto-docs/gusto.core.html#gusto.core.configuration.CompressibleParameters), except for `g` and `cp`. The sponge layer is set by passing in a `SpongeLayerParameters` object when we set up the compressible Euler equations. The parameters required to specify the sponge layer are: the height of the domain, the height at which the sponge layer begins and the strength of the damping.q

In [5]:
g = 9.80665              # acceleration due to gravity, in m/s^2
cp = 1004.               # specific heat capacity at constant pressure
parameters = CompressibleParameters(g=g, cp=cp)
sponge_depth = 10000.0   # depth of sponge layer, in m
sponge_mu = 0.15         # parameter for strength of sponge layer, in J/kg/K
sponge = SpongeLayerParameters(H=domain_height, z_level=domain_height-sponge_depth, mubar=sponge_mu/dt)
eqns = CompressibleEulerEquations(domain, parameters, sponge_options=sponge)

TypeError: EquationParameters.__init__() missing 1 required positional argument: 'mesh'

We will output the vertical component of the velocity as well as the perturbations to the potential temperature and density.

In [6]:
output = OutputParameters(dirname='nh_mountain', dumpfreq=10)
diagnostic_fields = [ZComponent('u'), Perturbation('theta'), Perturbation('rho')]
io = IO(domain, output, diagnostic_fields=diagnostic_fields)

We will set up a SemiImplicitQuasiNewton timestepper. This method splits the equation into terms that are treated explicitly and those that are treated semi-implicitly. Transport and forcing terms are treated separately. This allows for different time-steppers to be used for transporting the velocity and depth fields. We choose to use the trapezium rule for the velocity and an explicit strong stability preserving RK3 (SSPRK3) method for the density and potential temperature. We apply the DGUpwind method to the transport terms for all fields, and additionally use streamline upwind Petrov-Galerkin (SUPG) for the potential temperature.

In [7]:
theta_opts = SUPGOptions()
transported_fields = [TrapeziumRule(domain, "u"),
                      SSPRK3(domain, "rho"),
                      SSPRK3(domain, "theta", options=theta_opts)]
transport_methods = [DGUpwind(eqns, "u"),
                     DGUpwind(eqns, "rho"),
                     DGUpwind(eqns, "theta", ibp=theta_opts.ibp)]

linear_solver = CompressibleSolver(eqns)
stepper = SemiImplicitQuasiNewton(eqns, io,
                                  transported_fields,
                                  transport_methods,
                                  linear_solver=linear_solver)

NameError: name 'eqns' is not defined

The initial conditions for the potential temperature and density for this problem are in hydrostatic balance. Below we set the initial potential temperature in terms of the buoyancy frequency `N`.

In [8]:
# ensure we use the same parameter values for the initial conditions as we passed to the model
g = parameters.g
N = parameters.N

# get the coordinates so that we can set the vertical profile of the potential temperature
xz = SpatialCoordinate(mesh)
Tsurf = 300.
# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
thetab = Tsurf*exp(N**2*xz[1]/g)
Vt = domain.spaces('theta')
theta_b = Function(Vt).interpolate(thetab)

AttributeError: module 'gusto.solvers.parameters' has no attribute 'g'

We determine the Exner pressure using the `compressible_hydrostatic_balance` function which requires that we impose a boundary condition. As explained in the introduction, we know that the Exner pressure should be 1 on the flat bottom boundary but we cannot set this value as the boundary is now deformed. Instead we would like to set the value on the top boundary, which is still flat. To obtain the value of the Exner pressure at on the top boundary we iterate. First we set the Exner pressure to be 1 on the deformed lower boundary to get an estimate of the required value on the top boundary - this will be the minimum value of the Exner pressure that we have just computed. We now recompute the hydrostatic Exner pressure using this value on the top boundary. We can check the maximum value of the Exner pressure to see if it is close enough to 1 and if not, we can adjust the top boundary condition and recompute. We iterate this procedure until the Exner pressure is close enough to 1 on the bottom boundary. This then enables us to compute the density that is in hydrostatic balance with the prescribed potential temperature and we use those fields as the background reference profiles for the timestepper.

In [9]:
# set up functions for the initial hydrostatic profile of Exner pressure and density
Vr = domain.spaces('L2')
exner = Function(Vr)
rho_b = Function(Vr)

# Set up kernels to evaluate global minima and maxima of fields
min_kernel = MinKernel()
max_kernel = MaxKernel()

# First solve hydrostatic balance that gives Exner = 1 at bottom boundary
# This gives us a guess for the top boundary condition
exner_surf = 1
bottom_boundary = Constant(exner_surf, domain=mesh)
compressible_hydrostatic_balance(
    eqns, theta_b, rho_b, exner, top=False, exner_boundary=bottom_boundary
)

# Solve hydrostatic balance again, but now use minimum value from first
# solve as the *top* boundary condition for Exner
top_value = min_kernel.apply(exner)
top_boundary = Constant(top_value, domain=mesh)
logger.info(f'Solving hydrostatic with top Exner of {top_value}')
compressible_hydrostatic_balance(
    eqns, theta_b, rho_b, exner, top=True, exner_boundary=top_boundary
)

max_bottom_value = max_kernel.apply(exner)

# Now we iterate, adjusting the top boundary condition, until this gives
# a maximum value of 1.0 at the surface
lower_top_guess = 0.9*top_value
upper_top_guess = 1.2*top_value
max_iterations = 10
tolerance = 1e-7
for i in range(max_iterations):
    # If max bottom Exner value is equal to desired value, stop iteration
    if abs(max_bottom_value - exner_surf) < tolerance:
        break

    # Make new guess by average of previous guesses
    top_guess = 0.5*(lower_top_guess + upper_top_guess)
    top_boundary.assign(top_guess)

    compressible_hydrostatic_balance(
        eqns, theta_b, rho_b, exner, top=True, exner_boundary=top_boundary
    )

    max_bottom_value = max_kernel.apply(exner)

    # Adjust guesses based on new value
    if max_bottom_value < exner_surf:
        lower_top_guess = top_guess
    else:
        upper_top_guess = top_guess

# Perform a final solve to obtain hydrostatically balanced rho
compressible_hydrostatic_balance(
    eqns, theta_b, rho_b, exner, top=True, exner_boundary=top_boundary,
    solve_for_rho=True
)

stepper.set_reference_profiles([('rho', rho_b), ('theta', theta_b)])

NameError: name 'eqns' is not defined

The initial conditions for the potential temperature and density are the hydrostatically balanced fields calculated above. The initial condition for the wind is a $10ms^{-1}$ horizontal wind.

In [10]:
# Initial conditions
u0 = stepper.fields("u")
rho0 = stepper.fields("rho")
theta0 = stepper.fields("theta")

initial_wind = 10.0
u0.project(as_vector([initial_wind, 0.0]), bcs=eqns.bcs['u'])
rho0.assign(rho_b)
theta0.assign(theta_b)

NameError: name 'stepper' is not defined

Now we can run the simulation!

In [11]:
stepper.run(t=0, tmax=2*dt)

NameError: name 'stepper' is not defined